# Decision Trees

in the previous lesson we studied that k-NN is a model which predicts new values by modelling a piecewise-constant predictor. Each piece of the predictor is a function which implicitely maps a set of values to a decision region.  
An alternative to k-NN are **decision trees** which are complementary to k-NN in a way that they define regions too, but the way they are defined is different. Specifically, decision trees **explicitely** define decision regions based on rules applied to input variables.

The Basic idea behind a decision tree is to use a rule-based model organised as a binary tree.  
This binary tree partitions the input space $ R^p $ into subregions $ R_m $ which are disjointed $ R_m union R_{m'} = 0 \text{ for } m \ne m' $, and where $ M $ is the number of regions  

$ \mathbb{R}^p = \bigcup_{m=1}^{M} R_m $  

together with a piecewise-constant predictor  

$ \hat{y}(x) = c_m \text{ if } x \in{R_m} $


## Rules and tree structure

Disjoint regions are created by recursively applying a set of rules on the input features.  
We start from the root, where a rule is applied: $ x_j \leq t $ ? Where $ t $ is a treshold and $ x_j $ is an input features.  
If the condition is satisfied, a (left) Leaf Region is produced, which defines a decision region $ R_m $ where the function assumes a constant value $ c_m $, for example $ R_1 -> c_1 $.  
If the condition is not satisfied, the input is sent to the right child which can be an in *internal node* (with a new treshold value $ s $ for a new input feature $ x_k $) or a *leaf node* depending on the case.  

For each defined leaf region $ R_m $ we can identify the index set of training samples belonging to that region:  
$ S_m = \{ i : x_i \in R_m \} $ 
So for each leaf region the predictor produces a constant $ c_m $ value and this values only relies on the samples of the training set associated to the index set $ S_m $  

As we saw in the K-NN, the actual value of the constant $ c_m $ produced by the predictor depends on a calculation which varies from a classification or regression problem.  

For *regression*, the avarage value of the training outputs $ y_i $ in the ragion $ R_m $ is used.  
$ c_m = \frac{1}{|S_m|}\sum_{i\in S_m} y_i $  
For *classification*, the prediction is the most frequent class among the samples in $ S_m $, where $ n_{c}(S_m) $ denotes the number of samples in $ S_m $ belonging to class $ c $.  
$ c_m = \arg\max_{c \in \mathcal{C}} n_c(S_m). $

Rules are applied in order to let the decision tree split the input space in two smaller regions progressively. The rule acts a *geometric cut* which divides a region in a left region $ R_{L}(j,\leq t) $ and a right region $ R_{R}(j,t) $. From a geometric perspective, this operation corresponds to splitting the input space with a
hyperplane that is orthogonal to one of the coordinate axes. From a statistical perspective, it
separates the training samples into two groups, which are evaluated independently.  

The purpose of the split is to *reduce uncertainty* about the output variable. **A good split is one that produces child nodes in which the outputs are more concentrated, less variable, or more class-consistent than in the parent node**.  

So a question  can be arisen: *what is the best split we can choose?*  
The answer depends on the learning task. In both regression and classification, the guiding
principle is the same: a good split is one that produces child nodes that are more homogeneous,
or less uncertain, than the parent node.  In order to answer this question, we now introduce the concept of $ \text{ impurity } $ and we select the split that yelds the **largest reduction in impurity**.  

# Regression

In regression problems, output variables are numerical. So the concept of homogeneity within a node corresponds to having output values which are close to each other.  
A quantity which is able to describe the property of "closeness" of the output values is the Mean Square Error (MSE) which measures *how dispersed the outputs are around their mean*.  
For a given node of region $ R_m $ associated with an index set $ S $, the mean (avarage) output value is:  
$\bar{y}_S = \frac{1}{\lvert S\rvert} \sum_{i \in S} y_i$  
So we can measure how much each output value $ y_i $ is distant from the mean $ \bar{y}_S $, sum all the distances of each output and calculate the avarage value  
$\mathrm{MSE}(S)=\frac{1}{\lvert S\rvert}\sum_{i\in S}\left(y_i-\bar{y}_S\right)^2$  
The smaller the MSE value is, the closer and less dispersed the output values are within a node.  
   
## Impurity reduction
We can now evaluate the quality of a split by measuring the reduction of impurity caused by such split. Once a split is done we have two partitions of S, and we can calculate the weighetd avarage of the impurities (so the weighted avarage of the MSE of the two regions), where the wight is the value of $ \frac{| S_{L/R} |}{| S |} $, and sum the two weighted MSE in order to obtain the $ MSE_{post} $.  
The quality of the split is:  $ \Delta_{reg}(j,t) = MSE(S) - MSE_{post} $  

The **optimal split** is the one that maximizes $ \Delta_{reg}(j,t) $.


In [1]:
# Example
# consider the following dataset of one feature x_1 and a target variable y:

dataset = [
    (1.0, 1.2),
    (1.8, 1.8),
    (2.7, 2.6),
    (3.2, 3.9),
    (3.8, 4.1),
    (4.5, 4.8)
]

# we are dealing with numerical outputs, so this is a regression problem. the objective is to evaluate the quality of a split, for example, x_1 <= 2.5
# we need to calculate the MSE_post, which needs the MSE of each split. For the MSE we first need the mean/avarage output value:

Y_avg = sum(y for _, y in dataset) / len(dataset)
print("Average output value:", Y_avg)

MSE_Y = sum((y - Y_avg) ** 2 for _, y in dataset) / len(dataset)
print("MSE of the whole dataset:", MSE_Y)

# now let's apply the split and calculate the MSE_post, by the weighted average of the MSE of each split:
rule = 2.5
split_left = [(x, y) for x, y in dataset if x <= rule]
split_right = [(x, y) for x, y in dataset if x > rule]
Y_avg_left = sum(y for _, y in split_left) / len(split_left)
Y_avg_right = sum(y for _, y in split_right) / len(split_right)
MSE_left = sum((y - Y_avg_left) ** 2 for _, y in split_left) / len(split_left)
MSE_right = sum((y - Y_avg_right) ** 2 for _, y in split_right) / len(split_right)
MSE_post = (len(split_left) * MSE_left + len(split_right) * MSE_right) / len(dataset)
print("MSE after the split:", MSE_post)

# we can now calculate the impurity Delta:
Delta = MSE_Y - MSE_post
print("Impurity Delta:", Delta)

## Will print "Impurity Delta: 1.227222222222222"
# since Delta is positive, this means that the split has reduced the MSE, and thus it is a good split.

Average output value: 3.0666666666666664
MSE of the whole dataset: 1.6788888888888887
MSE after the split: 0.4516666666666666
Impurity Delta: 1.227222222222222


# Classification

In classification problems, output variables are categorical. So the concept of homogeneity within a node corresponds to having output values which are predominantely of a single class.  
A quantity which is able to describe the property of "predomincance" of the output values is the proportion $ p_{c}(S) $ of a sample belonging to the class $ c $ in a node $ S $, which measures *the most frequent/predominant class in a region*.  
For a given node of region $ R_m $ associated with an index set $ S $, the proportion value of a class $ c $ is:  
$\bar{y}_S = \frac{1}{\lvert S\rvert} \sum_{i \in S} y_i$  
The impurity measure quantifies how mixed the classes are within a node. We can use two measures for impurity: Gini and Entroy.  

## Gini
The Gini impurity is zero if all samples within a region belong to the same class, and increases as the class distribution becomes more uniform.  
$G(S)=1-\sum_{c\in\mathcal{C}}p_c(S)^2$  

## Entroy
Entropy measures the uncertainity associated with predicting the class label, and it as an information-theoretic interpretation.  
$H(S) = -\sum_{c \in C} p_c(S)\log_2 p_c(S)$  

## Impurity reduction   
We can now evaluate the quality of a split by measuring the reduction of impurity caused by such split. Once a split is done we have two partitions of S, and we can calculate the post-split impurity as the sum of the weighted avarage of the impurity measure - whichever Gini or Entropy is used - of each partition, obtaining the $ I_{post} $.  
The improvement associated with the split is:  $ \Delta_{cls}(j,t) = I(S) - I_{post} $  

The **optimal split** is the one that maximizes $ \Delta_{cls}(j,t) $.

In [ ]:
# Example
## Let's use a dataset consisting of 10 samples, class A = 5 and class B = 5, and a candidate split which produces: SL: A=3, B=1 and SR: A=2, B=4.
import numpy as np

dataset = [ 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B' ]
# first we calculate the proportion of each class in the whole dataset:
p_A = dataset.count('A') / len(dataset)
p_B = dataset.count('B') / len(dataset)
print("Proportion of class A:", p_A)
print("Proportion of class B:", p_B)
proportions = [p_A, p_B]

# Now we calculate Gini and Entropy for the whole dataset:  
Gini_Y = 1 - sum(p ** 2 for p in proportions)
Entropy_Y = -sum(p * np.log2(p) for p in proportions if p > 0)
print("Gini of the whole dataset:", Gini_Y)
print("Entropy of the whole dataset:", Entropy_Y)

# Now we apply the split and calculate Gini and Entropy for each split:
split_left = ['A', 'A', 'A', 'B']
split_right = ['A', 'A', 'B', 'B', 'B', 'B']
p_A_left = split_left.count('A') / len(split_left)
p_B_left = split_left.count('B') / len(split_left)
p_A_right = split_right.count('A') / len(split_right)
p_B_right = split_right.count('B') / len(split_right)
print("Proportion of class A in left split:", p_A_left)
print("Proportion of class B in left split:", p_B_left)
print("Proportion of class A in right split:", p_A_right)
print("Proportion of class B in right split:", p_B_right)
proportions_left = [p_A_left, p_B_left]
proportions_right = [p_A_right, p_B_right]

Gini_left = 1 - sum(p ** 2 for p in proportions_left)
Gini_right = 1 - sum(p ** 2 for p in proportions_right)
Entropy_left = -sum(p * np.log2(p) for p in proportions_left if p > 0)
Entropy_right = -sum(p * np.log2(p) for p in proportions_right if p > 0)
print("Gini of left split:", Gini_left)
print("Gini of right split:", Gini_right)
print("Entropy of left split:", Entropy_left)
print("Entropy of right split:", Entropy_right)

# Now we calculate the weighted average of Gini and Entropy for the splits:
Gini_post = (len(split_left) * Gini_left + len(split_right) * Gini_right) / len(dataset)
Entropy_post = (len(split_left) * Entropy_left + len(split_right) * Entropy_right) / len(dataset)
print("Gini after the split:", Gini_post)
print("Entropy after the split:", Entropy_post)

# Finally we calculate the impurity Delta for both Gini and Entropy:
Delta_Gini = Gini_Y - Gini_post
Delta_Entropy = Entropy_Y - Entropy_post
print("Impurity Delta for Gini:", Delta_Gini)
print("Impurity Delta for Entropy:", Delta_Entropy)

# We can observe that the imurity reduction by Gini is << 0.1, thus the split did not introduce any purity and it is better to leave the dataset unsplit.

Proportion of class A: 0.5
Proportion of class B: 0.5
Gini of the whole dataset: 0.5
Entropy of the whole dataset: 1.0
Proportion of class A in left split: 0.75
Proportion of class B in left split: 0.25
Proportion of class A in right split: 0.3333333333333333
Proportion of class B in right split: 0.6666666666666666
Gini of left split: 0.375
Gini of right split: 0.4444444444444444
Entropy of left split: 0.8112781244591328
Entropy of right split: 0.9182958340544896
Gini after the split: 0.41666666666666663
Entropy after the split: 0.8754887502163469
Impurity Delta for Gini: 0.08333333333333337
Impurity Delta for Entropy: 0.12451124978365313


# Growing a decision tree
A decision tree is constructed by recursively applying the splitting procedure described above.  
Algorithm (Decision Tree Growth).  
1. Start with all training samples at the root node.  
2. If a stopping condition is satisfied, create a leaf and assign a prediction.  
3. Otherwise, evaluate all candidate splits and select the one that maximizes impurity reduction.  
4. Partition the data according to the selected split and repeat the procedure on each child node.  

This recursive process continues until all leaves satisfy a stopping criterion.

## Stopping criteria and pruning
Without a stopping criterion, the complexity of the decision tree increases, which produces perfect training accuracy (each individual training sample consists of a leaf) but lacks generalisation.  
We can apply different stopping criterion, mainly consisting of two types: pre-pruning and post-pruning.  The aim is to *reduce variance and improve generalisation*.

### Pre-pruning
Pre-pruning consists in applying rules during the decision tree generation in order to stop the splitting. Several rules can be applied, and we stop the splitting at a node associatecd with index set $ S $ if at least one of the following conditions holds.  
1. **Pure node**, a node is pure when uncertainty about the predicted output is zero. For *regression* it corresponds to a 0 variance (MSE) for a given node; for *classification* it corresponds to an impurity index equals to 0, which is equivalent to a proportion of exactly 1 for a given class $ c $.  
2. **Too few samples**, we can define a treshold value of minimum required samples in order to split, or a *minimum leaf size* required in order to split.  
3. **Maximum depth reached**, we can stop the splitting when we reach a specific value $ d_{max} $ of depth for a node in the tree $ depth(S) $.  
4. **No worthwile improvement**, for every candidate split $ (j,t) $ we can compute the impurity reduction $ \Delta(j,t) $ and check if a minimum improvement $ \epsilon $ is reached.  
5. **No valid split exists**, Stop if all features are constant on the samples in S, or if every candidate split creates an empty child. 
These criteria control model complexity during training and reduce the risk of overfitting by
preventing the tree from fitting spurious, small-scale patterns in the training set.

### Post-pruning
In this case we let the tree grow to the end first, and then simplify by removing branches (decisions) that d not improve performance on validation data.